In [3]:
from Helper2 import *
from sympy import *

fRead = open('homework3SampeInput.txt')
myInputEqn = fRead.readline().strip()
myEqn = myInputEqn.split('=')

#myCompunds[0] is reactant, [1] is product or rigt side
myCompunds = [eachSide.split('+') for eachSide in myEqn]

reactantList =  [atomCount (eachComp) for eachComp in myCompunds[0]]
productList = [atomCount (eachComp) for eachComp in myCompunds[1]]
productListNeg = [{atom:-qty  for atom,qty in nextComp.items()} for nextComp in productList]

combinedList = reactantList + productListNeg
print ('Reactant, Product, Combined:\n', reactantList, "\n", productListNeg,"\n", combinedList )

mySymbols = symbols('s0:%d'%(  len(myCompunds[0]) + len(myCompunds[1]) ))
distAtomList = list( set( [atom for nextComp in reactantList for atom,qty in nextComp.items()] ))
print ("\nEqn Symbols:", mySymbols, "\nDistinct Atoms:", distAtomList)

#construct the #distAtomList linear equations
equation = []
i = 0 
for atom in distAtomList:
    equation.append(0)
    for x in combinedList:
        if atom in x:
            equation[i] = equation[i] + (mySymbols[combinedList.index(x)] * x[atom])
    i = i + 1
                              
solnNonFreeVar = (solve (equation, mySymbols) )
print("Equation:", equation, "\n\nSolution:", solnNonFreeVar)

#find lcm, make soln whole, add free variables, get final coeff, print result & done
leastMultiple = lcm([val.as_numer_denom()[1] for key,val in solnNonFreeVar.items()])  
#print ("Least Multiple", leastMultiple)

solnWhole = {k:leastMultiple*v for k,v in solnNonFreeVar.items()}
#print ("non-free:", solnWhole)

solnWholeSymbol = [k for k,v in solnWhole.items()]
solnWholeExpr =   [v for k,v in solnWhole.items()]

for nextSymbol in mySymbols:  
  solnWholeExpr   = [nextSoln.subs(nextSymbol, 1  ) for nextSoln in solnWholeExpr ]
finalResult = {nextSymbol:intValue for nextSymbol, intValue in zip(solnWholeSymbol, solnWholeExpr) }

#For free variables, set val = leastMultiple, as done in the 2nd parameter below
for nextSymbol in mySymbols:
    finalResult[nextSymbol] = finalResult.get(nextSymbol,leastMultiple)
print ("Final Coeff:", finalResult)

#print the balanced equation
total = myCompunds[0] + myCompunds[1]
balancedEqn, filler = "",""
for idx in range(len(mySymbols)):
  if idx == len(myCompunds[0]): filler = " = " 
  key = mySymbols[idx]
  if key in finalResult:
     balancedEqn += filler + str(finalResult[key]) + total[idx] 
  filler = "+" 
print ("\nInput and Balanced Eqns:\n", myInputEqn, "\n", balancedEqn)


Reactant, Product, Combined:
 [{'Hg': 1, 'O': 2, 'H': 2}, {'H': 3, 'P': 1, 'O': 4}] 
 [{'Hg': -3, 'P': -2, 'O': -8}, {'H': -2, 'O': -1}] 
 [{'Hg': 1, 'O': 2, 'H': 2}, {'H': 3, 'P': 1, 'O': 4}, {'Hg': -3, 'P': -2, 'O': -8}, {'H': -2, 'O': -1}]

Eqn Symbols: (s0, s1, s2, s3) 
Distinct Atoms: ['P', 'Hg', 'H', 'O']
Equation: [s1 - 2*s2, s0 - 3*s2, 2*s0 + 3*s1 - 2*s3, 2*s0 + 4*s1 - 8*s2 - s3] 

Solution: {s2: s3/6, s0: s3/2, s1: s3/3}
Final Coeff: {s2: 1, s0: 3, s1: 2, s3: 6}

Input and Balanced Eqns:
 Hg(OH)2  + H3PO4  = Hg3(PO4)2 + H2O 
 3Hg(OH)2  +2 H3PO4   = 1 Hg3(PO4)2 +6 H2O
